In [2]:
import sys
sys.path.append('../utils')

import numpy as np
from sklearn.linear_model import LinearRegression

from utils.basic_conformal_inference import ConformalPredictor

In [7]:
# Example usage
def dgp(n):
    X = np.random.randn(n, 1)
    Y = 2 * X.squeeze() + 1 + np.random.randn(n)
    return X, Y

n = 500
np.random.seed(0)
X, Y = dgp(n)
X_new = np.array([0.25, .54]).reshape(-1, 1)
Y_trial = np.linspace(-10, 10, 1000)
alpha = 0.1

predictor = ConformalPredictor(LinearRegression(), alpha)
predictor.fit(X, Y)
prediction_intervals = predictor.predict(X_new, Y_trial)
print(prediction_intervals)


[array([-0.17017017,  3.03303303]), array([0.39039039, 3.59359359])]


In [8]:
def monte_carlo(n_mc, n_sample, alpha):
    coverage = 0
    for _ in range(n_mc):
        X, Y = dgp(n_sample)
        X_new = np.array([X[-1]])
        y_new = Y[-1]
        X, Y = X[:-1], Y[:-1]
        Y_trial = np.linspace(np.min(Y) - 2, np.max(Y) + 2, 1000)
        
        predictor = ConformalPredictor(LinearRegression(), alpha)
        predictor.fit(X, Y)
        prediction_intervals = predictor.predict(X_new, Y_trial)
        coverage += (y_new >= min(prediction_intervals[0])) and (y_new <= max(prediction_intervals[0]))
    return coverage / n_mc

n_mc = 500
n = 5000
alpha = 0.1
coverage = monte_carlo(n_mc, n, alpha)

In [9]:
print(coverage)

0.882
